## Symbolic analysis of models

In this notebook, we use symbolic mathematics to study energy-based PySB model. We derive steady-state analytical solutions to study reaction networks without costly ODE simulations. First, we load the previously developed models for RAF and RAF inhibition:

In [1]:
#import complex_example_asym_RAF_two_RAFi as model
import toy_example_RAF_RAFi as model
from pysb.bng import generate_equations
from util_display import display_model_info

#generate the model equations 
model=model.model
generate_equations(model)

#display model informations
display_model_info(model)

Model information
Species: 6
Parameters: 12
Expressions: 8
Observables: 8
Total Rules: 2
Energy Rules: 2
Non-energy Rules: 0
Energy Patterns: 4
Reactions: 12


Next, we define the sympy systems of equations corresponding to the ODE system but with the left-hand side (the derivative definition) set to zero, meaning that the system is considered to be at steady-state: 

In [2]:
import sympy
import scipy

#create a list of expressions to be substituted into kinetic rates of reactions
species = sympy.Matrix([sympy.Symbol(f"s_{i}") for i in range(len(model.species))])
subs = {e: e.expand_expr() for e in model.expressions | model._derived_expressions}
subs.update({sympy.Symbol(f"__s{i}"): s for i, s in enumerate(species)}) 
kinetics = sympy.Matrix([r['rate'] for r in model.reactions]).xreplace(subs)    
#simplyfy kinetic
kinetics.simplify()
sm = sympy.SparseMatrix(*model.stoichiometry_matrix.shape, model.stoichiometry_matrix.todok())
obs_matrix = scipy.sparse.lil_matrix(
    (len(model.observables), len(model.species)), dtype=int
)
for i, obs in enumerate(model.observables):
    obs_matrix[i, obs.species] = obs.coefficients
om = sympy.SparseMatrix(*obs_matrix.shape, obs_matrix.todok()) 
odes = sm * kinetics
observables = om * species

The following cell currently needs to be customized to your specific model. Define conservation of mass expressions for all monomers, and an expression you would like to solve for.

In [3]:
# Define conservation of mass expressions (each equal to zero).
conservation = sympy.Matrix([
    model.parameters["R_0"] - observables[0],
    model.parameters["I_0"] - observables[1],
])
# This is just R_BRAFmut_active_obs, but it could be any expression.
expression = observables[2]

Solve the combined system of the ODEs and conservation expressions for the list of symbols used in our desired expression. There may be multiple solutions

In [4]:
symbols = list(expression.atoms(sympy.Symbol))
system = sympy.Matrix.vstack(odes, conservation)
solutions = sympy.nonlinsolve(system, symbols)

Substitute each solution into our original expression, keeping only unique results (sometimes the solver returns multiple solutions that end up yielding the same final expression value).

In [5]:
sympy.Matrix(list(set(expression.xreplace(dict(zip(symbols, s))).simplify() for s in solutions)))

Matrix([[-I_0 + R_0 + s_1]])

Then, we define a last equations that calculates the amount of active RAFs based on the status of RAF dimerization and drug binding:

We use symbolic solvers to derive the analytical function for the amount of active RAFs at steady state: 